# Tensorflow Estimators

This document introduces `tf.estimator` - a high level Tensorflow API. Estimators encapsulate the following actions.

* Training
* Evaluation
* Prediction
* Export for serving

In [ ]:
import tempfile
import os

import tensorflow as tf
import tensorflow_datasets as tfds

# Steps to use the TF estimator APIs

1. Define dataset metadata
2. Define data input function to read the data from Pandas dataframe + apply feature processing
3. Create TF feature columns based on metadata + extended feature columns
4. Instantiate an estimator with required feature columns & parameters
5. Train estimator using training data
6. Evaluate estimator using test data
7. Perform predictions
8. Save & serve the estimator

# Example

We are going to train a deep neural network classifier with the below structure. All input and output values will be `float32`, and the sum of the output values will be 1 ( as we are predicting the probability for each individial iris_type)

## Representing our dataset

1. Write an input functions.

`input_fn` should return a `tf.data.Dataset` that yields pairs in that format.

For example, the following code builds a `tf.data.Dataset` from the Titanic dataset's train.csv file.

In [ ]:
def train_input_fn():
    titanic_file = tf.keras.utils.get_file('train.csv', "https://storage.googleapis.com/tf-datasets/titanic/train.csv")
    titanic = tf.data.experimental.make_csv_dataset(
        titanic_file, batch_size=32, label_name='survived'
    )
    
    titanic_batches = (
        titanic.cache().repeat().shuffle(500).prefetch(tf.data.AUTOTUNE)
    )
    
    return titanic_batches


2. Define feature columns

Each tf.feature_column identifies a feature name, its type and any input pre-processing.

For example, the following snippet creates three feature columns

* The first uses the `age` feature directly as a floating-point input.
* The scond usees the `class` featue as a categorical input
* The third uses the `embark_town` as a categorical input, but uses the hashing trick to avoid the need to enumerate the options and to set the number of options.

In [ ]:
age = tf.feature_column.numeric_column('age')
cls = tf.feature_column.categorical_column_with_vocabulary_list('class',['First','Second','Third'])
embark = tf.feature_column.categorical_column_with_hash_bucket('embark_town',32)

3. Instantiate the relevant pre-made estimator

For example, here's a sample instantiation of pre-made Estimator named `LinearClassifier`:

In [ ]:
model_dir = tempfile.mkdtemp()
model = tf.estimator.LinearClassifier(
    model_dir = model_dir,
    feature_columns = [embark, cls, age],
    n_classes = 2
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp3d6qb_r6', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
model = model.train(input_fn=train_input_fn, steps=100)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
40960/30874 [=======================================] - 0s 0us/step


2021-12-10 17:45:26.518547: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-10 17:45:26.519117: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-10 17:45:26.524041: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-10 17:45:26.524590: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-10 17:45:26.525210: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/home/puneet/Envs/venv/lib/python3.9/site-packages/tensorflow_estimator/python/estimator/canned/linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.


2021-12-10 17:45:27.085585: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-10 17:45:27.212590: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-10 17:45:27.212977: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-10 17:45:27.213395: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


2021-12-10 17:45:27.753003: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-10 17:45:27.753417: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-10 17:45:27.753886: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-10 17:45:27.754233: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-10 17:45:27.754685: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp3d6qb_r6/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.6931472, step = 0
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 100...
INFO:tensorflow:Saving checkpoints for 100 into /tmp/tmp3d6qb_r6/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 100...
INFO:tensorflow:Loss for final step: 0.55999947.


2021-12-10 17:45:28.674125: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [ ]:
result = model.evaluate(train_input_fn, steps=10)

for key, value in result.items():
  print(key, ":", value)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-12-10T17:45:52
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp3d6qb_r6/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


2021-12-10 17:45:52.313230: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-10 17:45:52.313739: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-10 17:45:52.314223: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-10 17:45:52.314629: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-10 17:45:52.315083: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

INFO:tensorflow:Evaluation [1/10]
INFO:tensorflow:Evaluation [2/10]
INFO:tensorflow:Evaluation [3/10]
INFO:tensorflow:Evaluation [4/10]
INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Inference Time : 0.34888s
INFO:tensorflow:Finished evaluation at 2021-12-10-17:45:52
INFO:tensorflow:Saving dict for global step 100: accuracy = 0.653125, accuracy_baseline = 0.584375, auc = 0.7120542, auc_precision_recall = 0.6178574, average_loss = 0.62306327, global_step = 100, label/mean = 0.415625, loss = 0.62306327, precision = 0.71153843, prediction/mean = 0.34650087, recall = 0.2781955
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 100: /tmp/tmp3d6qb_r6/model.ckpt-100
accuracy : 0.653125
accuracy_baseline : 0.584375
auc : 0.7120542
auc_precision_recall : 0.6178574
average_loss : 0.62306327
label/mean : 0.41562

2021-12-10 17:45:52.630230: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
